# Agente Pokémon con **LangGraph** + **Memoria**

Notebook que construye un agente ReAct con LangGraph, integrando:
- Tools basadas en PokéAPI
- Memoria con checkpoint (thread_id)
- Instrucciones en español y tabla Markdown para comparaciones


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import Optional
import re, requests

from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

BASE = "https://pokeapi.co/api/v2"
def _norm(s: str) -> str:
    """Normaliza nombres para PokéAPI (Mr. Mime -> mr-mime)."""
    return s.strip().lower().replace(" ", "-").replace(".", "").replace("’", "").replace("'", "")


In [ ]:
@tool("get_pokemon_info")
def get_pokemon_info(name: str):
    """Devuelve tipos, stats, altura y peso de un Pokémon por nombre."""
    r = requests.get(f"{BASE}/pokemon/{_norm(name)}", timeout=20); r.raise_for_status()
    d = r.json()
    return {
        "name": d["name"],
        "id": d.get("id"),
        "types": [t["type"]["name"] for t in d["types"]],
        "stats": {s["stat"]["name"]: s["base_stat"] for s in d["stats"]},
        "height": d.get("height"),
        "weight": d.get("weight"),
    }

def _walk_chain(chain):
    paths = []
    def dfs(node, path):
        curr = node['species']['name']
        new_path = path + [curr]
        if not node['evolves_to']:
            paths.append(new_path)
        else:
            for nxt in node['evolves_to']:
                dfs(nxt, new_path)
    dfs(chain, [])
    return paths

@tool("get_evolution_paths")
def get_evolution_paths(name: str):
    """Devuelve la cadena de evolución completa de un Pokémon (rutas)."""
    s = requests.get(f"{BASE}/pokemon-species/{_norm(name)}", timeout=20); s.raise_for_status()
    evo_url = s.json()['evolution_chain']['url']
    e = requests.get(evo_url, timeout=20); e.raise_for_status()
    return {"paths": _walk_chain(e.json()['chain'])}

@tool("get_type_info")
def get_type_info(type_name: str):
    """Devuelve fortalezas y debilidades del tipo (damage relations)."""
    r = requests.get(f"{BASE}/type/{_norm(type_name)}", timeout=20); r.raise_for_status()
    d = r.json(); dmg = d['damage_relations']; names = lambda xs: [x['name'] for x in xs]
    return {
        "name": d['name'],
        "double_damage_to": names(dmg['double_damage_to']),
        "half_damage_to": names(dmg['half_damage_to']),
        "no_damage_to": names(dmg['no_damage_to']),
        "double_damage_from": names(dmg['double_damage_from']),
        "half_damage_from": names(dmg['half_damage_from']),
        "no_damage_from": names(dmg['no_damage_from'])
    }

@tool("get_move_info")
def get_move_info(move_name: str):
    """Devuelve detalles de un movimiento (poder, precisión, tipo, PP, efecto)."""
    r = requests.get(f"{BASE}/move/{_norm(move_name)}", timeout=20); r.raise_for_status()
    d = r.json()
    name_es = next((n['name'] for n in d.get('names', []) if n['language']['name']=='es'), d['name'])
    desc_es = next((e['effect'] for e in d.get('effect_entries', []) if e['language']['name']=='es'),
                   next((e['effect'] for e in d.get('effect_entries', []) if e['language']['name']=='en'), None))
    return {
        "name": name_es,
        "type": d['type']['name'],
        "power": d.get('power'),
        "accuracy": d.get('accuracy'),
        "pp": d.get('pp'),
        "damage_class": d['damage_class']['name'] if d.get('damage_class') else None,
        "effect": desc_es
    }

class CompareArgs(BaseModel):
    """Argumentos para comparar dos Pokémon.

    - Opción A: `pokemon1` y `pokemon2` por separado.
    - Opción B: `pair` con ambos nombres en una cadena ("Pikachu, Raichu" o "Pikachu y Raichu").
    """
    pokemon1: Optional[str] = Field(None)
    pokemon2: Optional[str] = Field(None)
    pair: Optional[str] = Field(None)

def _split_pair(text: str):
    if not text:
        return (None, None)
    t = re.sub(r"\s+y\s+", ",", text.strip(), flags=re.IGNORECASE)
    parts = [p.strip() for p in t.split(",") if p.strip()]
    if len(parts) >= 2:
        return parts[0], parts[1]
    return (None, None)

@tool("compare_pokemon", args_schema=CompareArgs)
def compare_pokemon(pokemon1: Optional[str] = None, pokemon2: Optional[str] = None, pair: Optional[str] = None):
    """Compara stats base de dos Pokémon y devuelve filas y resumen de ganadores."""
    if (pokemon1 and not pokemon2) and ("," in pokemon1 or " y " in pokemon1.lower()):
        pokemon1, pokemon2 = _split_pair(pokemon1)
    if (not pokemon1 or not pokemon2) and pair:
        pokemon1, pokemon2 = _split_pair(pair)
    if not pokemon1 or not pokemon2:
        return {"error": "Proporciona dos nombres, ej.: pokemon1='Pikachu', pokemon2='Raichu' o pair='Pikachu, Raichu'."}
    d1 = get_pokemon_info(pokemon1); d2 = get_pokemon_info(pokemon2)
    name1, s1 = d1.get('name', pokemon1), d1.get('stats', {})
    name2, s2 = d2.get('name', pokemon2), d2.get('stats', {})
    all_stats = sorted(set(s1.keys()) | set(s2.keys()))
    rows, winners = [], {}
    for stat in all_stats:
        v1, v2 = s1.get(stat), s2.get(stat)
        win = None if (v1 is None or v2 is None) else (name1 if v1 > v2 else (name2 if v2 > v1 else 'Empate'))
        rows.append({"stat": stat, name1: v1, name2: v2, "winner": win})
        winners[stat] = win
    c1 = sum(1 for w in winners.values() if w == name1)
    c2 = sum(1 for w in winners.values() if w == name2)
    summary = {"best_overall": name1 if c1 > c2 else (name2 if c2 > c1 else 'Empate'),
               "wins": {name1: c1, name2: c2, "draws": sum(1 for w in winners.values() if w == 'Empate')}}
    return {"pokemon": [name1, name2], "comparison": rows, "summary": summary}


In [ ]:
TOOLS = [get_pokemon_info, get_evolution_paths, get_type_info, get_move_info, compare_pokemon]

# LLM con tool calling (LangGraph usa .bind_tools)
llm = ChatOpenAI(model="gpt-4o-mini").bind_tools(TOOLS)

SYSTEM_INSTRUCTION = (
    "Responde SIEMPRE en español.\n\n"
    "Cuando la herramienta `compare_pokemon` devuelva resultados, presenta:\n"
    "1) Una **tabla Markdown** con columnas: Stat | Valor de Pokémon 1 | Valor de Pokémon 2 | Ganador.\n"
    "2) Un **resumen** indicando cuál Pokémon es mejor en general y cuántos stats ganó cada uno."
)

def agent_node(state: MessagesState):
    """Nodo principal del agente: añade system prompt y pide al LLM el siguiente paso."""
    msgs = state["messages"]
    system = {"role": "system", "content": SYSTEM_INSTRUCTION}
    response = llm.invoke([system] + msgs)
    return {"messages": [response]}

tool_node = ToolNode(TOOLS)

def route_after_agent(state: MessagesState):
    last = state["messages"][ -1 ]
    # Si el modelo pidió tools → ir a tools; si no → terminar
    return "tools" if getattr(last, "tool_calls", None) else END

graph = StateGraph(MessagesState)
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)
graph.add_edge("tools", "agent")
graph.add_conditional_edges("agent", route_after_agent, {"tools": "tools", END: END})
graph.set_entry_point("agent")

# Memoria en RAM (checkpointing). Para persistir entre sesiones, se puede usar SQLite.
memory = MemorySaver()
app = graph.compile(checkpointer=memory)
print("✅ Grafo compilado con memoria (checkpointing)")


In [ ]:
# Ejemplo de uso con memoria por thread_id
config = {"configurable": {"thread_id": "pokemon-demo-1"}}

# Turno 1
result1 = app.invoke({"messages": [("user", "¿De qué tipo es Gengar y contra qué tipos es fuerte o débil?")]}, config)
print(result1["messages"][-1].content)

# Turno 2 (usa el contexto previo del hilo)
result2 = app.invoke({"messages": [("user", "Ahora compáralo con Alakazam en velocidad y ataque")]} , config)
print(result2["messages"][-1].content)
